# Classify Category (Decision Tree)
##### Mr.Jakkrit Sonsanit

***

### Import Library 

In [1]:
import pandas as pd
import numpy as np
from pythainlp.corpus.common import thai_words
from pythainlp.tokenize import dict_trie, word_tokenize
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import cross_val_score

### Import data

In [2]:
df = pd.read_csv("Expenses.csv")
df.tail()

,date,text,money,cate
176,2019-11-04,ข้าวเที่ยง,50.0,Food
177,2019-11-04,ชานม,40.0,Food
178,2019-11-04,ข้าวเย็น,40.0,Food
179,2019-11-04,หมาล่า,30.0,Food
180,2019-11-05,ค่าอิเทอร์เน็ต,1262.0,Bill


### Create Corpus

In [3]:
custom_dict = set(thai_words())
word = ['ราเมง', 'อิเกีย', 'คาปูชิโน่', 'น้ำมัน','หอยลาย', 'ปุ้มปุ้ย']
for i in word:
    custom_dict.add(i)
trie = dict_trie(dict_source=custom_dict)

corpus = []
for i in df.text:
    for j in word_tokenize(i, engine='dict', custom_dict=trie):
        if j not in corpus:
            corpus.append(j)
corpus
print(corpus)

['นม', 'ช็อคโกแลต', 'ข้าว', 'เที่ยง', 'น้ำ', 'กระเจี๊ยบ', 'ชา', 'ข้าวเย็น', 'ค่า', 'อินเตอร์เน็ต', 'ค่าโทรศัพท์', 'หมา', 'ล่า', 'ไอติม', 'ตัดผม', 'ซาลาเปา', 'ไส้กรอก', 'ดอย', 'คำ', 'ใบมีดโกน', 'ห้อง', 'Netflix', 'เงินเก็บ', 'ราเมง', 'นมเปรี้ยว', 'ขนมปัง', 'ขนม', 'arl', 'mrt', 'ซื้อ', 'กางเกง', 'ถั่ว', 'มาม่า', 'อิชิตัน', 'คาปูชิโน่', 'viu', 'บัวลอย', 'ค่ารถ', 'โกโก้', 'หนัง', 'เติม', 'เกม', 'ดู', 'โอ้', 'เด้ง', 'ป็อป', 'คอน', 'เหล้า', 'กระ', 'เพรา', 'หอมมะลิ', 'หอยลาย', 'ปุ้มปุ้ย', 'พริก', 'แกง', 'หมู', 'น้ำเปล่า', 'กระทะ', 'เสื้อ', 'กระเป๋า', 'อิเกีย', 'แท๊กซี่', 'ลูกชิ้น', 'มะละกอ', 'นวด', 'ติ๋ม', 'ซำ', 'ธูป', 'เทียน', 'ซักผ้า', ' ', 'spotify', 'สอง', 'แถว', '2', 'ก๋วยเตี๋ยว', 'ปลากระป๋อง', 'กับข้าว', 'มะตูม', 'หวาน', 'น้ำเต้าหู้', 'โอ', 'BTS', 'ตุ้มหู', 'ไข่ลวก', 'ทูน่า', 'ปลา', 'แอปเปิ้ล', 'ยำ', 'รถตู้', 'bts', 'อิ', 'เท', 'อร', '์', 'เน็ต']


### Create Bag of Word

In [4]:
BOW = [list() for i in range(len(df.text))]
l = 0
count = 1
for i in df.text:
    tmp = word_tokenize(i, engine='dict', custom_dict=trie)
    for j in corpus:

        if j in tmp:

            BOW[l].append(tmp.count(j))
            tmp.remove(j)
            
        else:
            BOW[l].append(0)
        
    if len(tmp) != 0:
        BOW[l].append(len(tmp))
    elif len(tmp) == 0:
        BOW[l].append(0)
    l += 1

### Train Decision Tree model

In [5]:
ytarget = df.cate
xtrain = BOW

dtree = DecisionTreeClassifier()
dtree.fit(X=xtrain,y=ytarget)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

### Cross Validation 5 fold

In [6]:
cross_val_score(dtree, xtrain, ytarget, cv=5, scoring="accuracy").mean()

/Users/jakkrit/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


0.8635349686433276

### Create Bag of Word form Test data

In [13]:
txt = ['หมูกระทะ']
BOW_t = [list() for i in range(len(txt))]
l = 0
for i in txt:
    tmp = word_tokenize(i, engine='dict', custom_dict=trie)
    for j in corpus:

        if j in tmp:

            BOW_t[l].append(tmp.count(j))
            tmp.remove(j)
        else:
            BOW_t[l].append(0)
        
    if len(tmp) != 0:
        BOW_t[l].append(len(tmp))
    elif len(tmp) == 0:
        BOW_t[l].append(0)
    l += 1

### Map Bag of word with corpus

In [14]:
# corpus_t = corpus.append('Other')
# ch = pd.DataFrame({
#     'train':corpus,
#     'target':BOW_t[0]
# })
# ch

### Predict Category 

In [15]:
predictiontree = dtree.predict(BOW_t)
predictiontree

array(['Food'], dtype=object)